In [ ]:
import requests
import json
import re

# Define API URL
API_URL = "http://172.19.0.1:11434/api/chat"

# Read prompts from text files
def read_file(filename):
    try:
        with open(filename, "r", encoding="utf-8") as file:
            return file.read().strip()
    except FileNotFoundError:
        print(f"Error: {filename} not found.")
        return None

# Get system and user prompts from text files
system_prompt = read_file("prompts/labelling/labelling_promptv3.txt")
user_prompt = read_file("prompts/class_0_sample.txt")

# Check if both files are successfully read
if system_prompt is None or user_prompt is None:
    exit("Missing input files. Please check system_prompt.txt and user_prompt.txt.")

# Define the JSON payload
payload = {
    "model": "deepseek-r1:32b",
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    "options": {
        "seed": 111,
        "temperature": 0.4,
        "num_ctx": 10240,
        "stream_response": False
    },
    "stream": False
}

# Send the request
try:
    response = requests.post(API_URL, json=payload)
    response.raise_for_status()  # Raise error if request fails
    response_data = response.json()

    # Extract message content
    message_content = response_data.get("message", {}).get("content", "")

    # Extract the <think> reasoning part
    thought_match = re.search(r"<think>\n(.*?)\n</think>", message_content, re.DOTALL)
    thought = thought_match.group(1).strip() if thought_match else "No thought found."

    # Extract the JSON output
    json_match = re.search(r"```json\n(.*?)\n```", message_content, re.DOTALL)
    output = json.loads(json_match.group(1)) if json_match else {}

    # Save to JSON file
    # output_data = {
    #     "output": output,
    #     "thought": thought
    # }
    output['thought'] = thought
    # with open("response_output.json", "w", encoding="utf-8") as json_file:
    #     json.dump(output_data, json_file, indent=4)

    # print("Parsed response saved to response_output.json")

except requests.exceptions.RequestException as e:
    print("Error sending request:", e)
except json.JSONDecodeError as e:
    print("Error decoding JSON:", e)


In [ ]:
output